Celeb-A 데이터 셋을 사용한 VAE 네트워크 구조는 이전 MNIST 손글씨 모델과 다음이 다르다.

1. 이 데이터셋의 입력 채널은 RGB이므로 3개이다. 따라서 디코더의 마지막에 있는 Conv2DTranspose층의 채널 수를 변경

2. 사용하는 latent_space의 차원 수는 2개에서 200개로 늘린다. 얼굴이 숫자 이미지보다 훨씬 복잡하기에 이미지에 있는 상세 정보를 충분히 encoding 하기 위해.

3. 훈련 속도를 높이기 위해 각 Conv2D층 뒤에 BatchNormalization층을 추가한다. 각 batch에 대한 실행 속도는 더 걸리더라도, 동일한 수준의 손실에 도달하기 위한 배치 횟수는 크게 줄어든다. (BN, Convolution, Activation 층에 순서에 대한 부분은 논외)

4. Dropout층 추가

5. 폴더에 있는 이미지 수가 많으므로 python generator를 사용해 VAE로 주입한다. batch 단위로 훈련하기 때문에 사전에 이미지를 메모리에 모두 로드할 필요가 없다. Keras의 내장 메서드인 fit_generator를 사용한다.

In [1]:
import tensorflow
import os
from glob import glob
import numpy as np

from models.VAE import VariationalAutoencoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tensorflow.compat.v1.disable_eager_execution()

In [2]:
# run params
section = 'vae'
run_id = '0001'
data_name = 'faces'
RUN_FOLDER = 'run/{}/'.format(section)
RUN_FOLDER += '_'.join([run_id, data_name])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #


DATA_FOLDER = './data/celeb/'

In [3]:
### Data Load
INPUT_DIM = (128,128,3)
BATCH_SIZE = 32

filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.jpg')))

NUM_IMAGES = len(filenames)

In [4]:
data_gen = ImageDataGenerator(rescale=1./255)

data_flow = data_gen.flow_from_directory(DATA_FOLDER
                                         , target_size = INPUT_DIM[:2] # integer tuple(height, width)
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         , class_mode = 'input' # input과 동일한 이미지(주로 오토인코더와 같이 사용)
                                         , subset = "training"
                                            )

Found 202599 images belonging to 1 classes.


In [5]:
### Model Build
vae = VariationalAutoencoder(
                input_dim = INPUT_DIM
                , encoder_conv_filters=[32,64,64, 64]
                , encoder_conv_kernel_size=[3,3,3,3]
                , encoder_conv_strides=[2,2,2,2]
                , decoder_conv_t_filters=[64,64,32,3]
                , decoder_conv_t_kernel_size=[3,3,3,3]
                , decoder_conv_t_strides=[2,2,2,2]
                , z_dim=200
                , use_batch_norm=True
                , use_dropout=True)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
vae.encoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 32)   128         encoder_conv_0[0][0]             
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 64, 64, 32)   0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [7]:
vae.decoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 200)]             0         
_________________________________________________________________
dense (Dense)                (None, 4096)              823296    
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 16, 16, 64)        36928     
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 64)        0   

In [8]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 10000
EPOCHS = 50
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [9]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

In [10]:
%%time
vae.train_with_generator(     
    data_flow
    , epochs = EPOCHS
    , steps_per_epoch = NUM_IMAGES / BATCH_SIZE
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
6332/6331 [==============================] - ETA: 0s - loss: 273.9905 - vae_r_loss: 212.0313 - vae_kl_loss: 61.9549
Epoch 00001: saving model to run/vae/0001_faces/weights/weights-001-273.99.h5

Epoch 00001: saving model to run/vae/0001_faces/weights/weights.h5
6332/6331 [==============================] - 2047s 323ms/step - loss: 273.9905 - vae_r_loss: 212.0313 - vae_kl_loss: 61.9549 - lr: 5.0000e-04
Epoch 2/50
6331/6331 [============================>.] - ETA: 0s - loss: 233.6634 - vae_r_loss: 174.9031 - vae_kl_loss: 58.7604
Epoch 00002: saving model to run/vae/0001_faces/weights/weights-002-233.67.h5

Epoch 00002: saving model to run/vae/0001_faces/weights/weights.h5
6332/6331 [==============================] - 315s 50ms/step - loss: 233.6810 - vae_r_loss: 174.9069 - vae_kl_loss: 58.7604 - lr: 5.0000e-04
Epoch 3/50
6331/6331 [============================>.] - ETA: 0s - loss: 228.6415 - vae_r_loss: 1

6332/6331 [==============================] - 313s 50ms/step - loss: 219.7231 - vae_r_loss: 160.7375 - vae_kl_loss: 58.9821 - lr: 5.0000e-04
Epoch 20/50
6331/6331 [============================>.] - ETA: 0s - loss: 219.5744 - vae_r_loss: 160.5742 - vae_kl_loss: 59.0003
Epoch 00020: saving model to run/vae/0001_faces/weights/weights-020-219.58.h5

Epoch 00020: saving model to run/vae/0001_faces/weights/weights.h5
6332/6331 [==============================] - 314s 50ms/step - loss: 219.5794 - vae_r_loss: 160.5751 - vae_kl_loss: 59.0005 - lr: 5.0000e-04
Epoch 21/50
6332/6331 [==============================] - ETA: 0s - loss: 219.5032 - vae_r_loss: 160.5015 - vae_kl_loss: 58.9955
Epoch 00021: saving model to run/vae/0001_faces/weights/weights-021-219.50.h5

Epoch 00021: saving model to run/vae/0001_faces/weights/weights.h5
6332/6331 [==============================] - 315s 50ms/step - loss: 219.5032 - vae_r_loss: 160.5015 - vae_kl_loss: 58.9955 - lr: 5.0000e-04
Epoch 22/50
6332/6331 [=========

6331/6331 [============================>.] - ETA: 0s - loss: 218.4559 - vae_r_loss: 159.3837 - vae_kl_loss: 59.0723
Epoch 00039: saving model to run/vae/0001_faces/weights/weights-039-218.46.h5

Epoch 00039: saving model to run/vae/0001_faces/weights/weights.h5
6332/6331 [==============================] - 314s 50ms/step - loss: 218.4554 - vae_r_loss: 159.3837 - vae_kl_loss: 59.0721 - lr: 5.0000e-04
Epoch 40/50
6331/6331 [============================>.] - ETA: 0s - loss: 218.4352 - vae_r_loss: 159.3737 - vae_kl_loss: 59.0614
Epoch 00040: saving model to run/vae/0001_faces/weights/weights-040-218.44.h5

Epoch 00040: saving model to run/vae/0001_faces/weights/weights.h5
6332/6331 [==============================] - 315s 50ms/step - loss: 218.4351 - vae_r_loss: 159.3736 - vae_kl_loss: 59.0616 - lr: 5.0000e-04
Epoch 41/50
6332/6331 [==============================] - ETA: 0s - loss: 218.3032 - vae_r_loss: 159.2509 - vae_kl_loss: 59.0421
Epoch 00041: saving model to run/vae/0001_faces/weights/